# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [33]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [34]:
data=pd.read_csv("../weatherPy/weather_data.csv")
data.drop('Unnamed: 0',axis=1,inplace=True)
data.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,kisangani,0.52,25.20,71.10,91.0,99.0,0.98,CD,1.595375e+09
1,bonavista,48.65,-53.11,64.44,76.0,68.0,8.25,CA,1.595375e+09
2,taoudenni,22.68,-3.98,102.11,11.0,0.0,14.03,ML,1.595375e+09
3,bethel,41.37,-73.41,84.20,48.0,1.0,5.82,US,1.595375e+09
4,punta arenas,-53.15,-70.92,23.00,92.0,9.0,2.24,CL,1.595375e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [35]:
gmaps.configure(api_key=g_key)
#configure gamps
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
center=(40.416775,-3.703790) #spain as the center
zoom_level=5
fig = gmaps.figure(layout=figure_layout,zoom_level=zoom_level,center=center)
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…

In [36]:
locations=[(lat,lng) for lat,lng in zip(data['Lat'],data['Lng'])]
humidity=data['Humidity']

In [37]:
heat_layer = gmaps.heatmap_layer(locations=locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

Narrow down the DataFrame to find your own ideal weather conditions. For example:
Maximum temperature lower than 80 degrees but higher than 70.
Wind speed less than 10 mph.
Zero cloudiness.
Note: Feel free to adjust to your specifications but be sure to limit the number of rows returned by your API requests to a reasonable number like 10.






In [38]:
conditions=(data['Max Temp']<75)&(data['Max Temp']>65)&(data['Wind Speed']<8)&(data['Cloudiness']<60)
hh_data=data[conditions]
print(f" {len(hh_data)} ideal options")
hh_data.head()

 39 ideal options


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
17,lata,40.16,-8.33,66.99,90.0,23.0,3.76,PT,1.595375e+09
64,ribeira grande,38.52,-28.70,69.39,75.0,22.0,3.69,PT,1.595375e+09
102,east london,-33.02,27.91,65.57,26.0,0.0,5.35,ZA,1.595375e+09
124,yinchuan,38.47,106.27,73.38,59.0,50.0,7.31,CN,1.595375e+09
148,xinqing,48.23,129.50,74.14,46.0,51.0,4.54,CN,1.595375e+09


In [46]:
#configure gamps
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout,center=center,zoom_level=zoom_level)
fig

locations=[(lat,lng) for lat,lng in zip(hh_data['Lat'],hh_data['Lng'])]
humidity=hh_data['Humidity']
heat_layer = gmaps.heatmap_layer(locations=locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [47]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
hotel_data={}


for location,city,country in zip(locations,hh_data['City'],hh_data['Country']):
    params = {
    "location": f'{location[0]}, {location[1]}',
    "rankby": "distance",
    "type": "hotel",
    "key": g_key}
    response = requests.get(base_url, params=params)
    data=response.json()
    for i in range(len(data['results'])):            
        name=data['results'][i]['name']
        lat=data['results'][i]["geometry"]['location']['lat']
        lng=data['results'][i]["geometry"]['location']['lng']
        hotel_data.update({name:[name,country,city,lat,lng]})

    

In [48]:
print(f'We have {len(hotel_data)} hotel options')

We have 645 hotel options


In [49]:
hotel_df=pd.DataFrame.from_dict(hotel_data, orient='index',columns=['Hotel Name','Country','City','Lat','Lng'])
hotel_df.head()

,Hotel Name,Country,City,Lat,Lng
Ra Auto MOTORSPORT,Ra Auto MOTORSPORT,PT,lata,40.160743,-8.333757
Marché de Semide,Marché de Semide,PT,lata,40.158119,-8.333762
Cemitério de Semide,Cemitério de Semide,PT,lata,40.157511,-8.333314
"Manoscortes, Comercio De Vestuario, Artigos Para O Lar E Papelaria, Lda.","Manoscortes, Comercio De Vestuario, Artigos Pa...",PT,lata,40.158879,-8.325524
Minipreço,Minipreço,PT,lata,40.158056,-8.334288


In [50]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [51]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
# Add the layer to the map
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…